LegalBERT Model Implementation for Classification (Without Strides max length 512 words)

In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset

/home/spyman/Documents/Deep Learning/LAW-AI classification/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('Input.csv')

label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Text'].values, df['Label'].values, test_size=0.2, random_state=42
)

tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=512)

/home/spyman/Documents/Deep Learning/LAW-AI classification/myenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
df['Label']

0      3
1      1
2      2
3      1
4      2
      ..
502    0
503    2
504    1
505    1
506    1
Name: Label, Length: 507, dtype: int64

In [3]:
class LegalCaseDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = LegalCaseDataset(train_encodings, train_labels)
val_dataset = LegalCaseDataset(val_encodings, val_labels)

In [4]:
model = BertForSequenceClassification.from_pretrained('nlpaueb/legal-bert-base-uncased', num_labels=len(label_encoder.classes_))

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy='epoch',
    load_best_model_at_end=True
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/spyman/Documents/Deep Learning/LAW-AI classification/myenv/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
trainer.train()

  4%|▍         | 10/255 [01:51<44:04, 10.79s/it]

{'loss': 1.4504, 'grad_norm': 9.02530574798584, 'learning_rate': 4.803921568627452e-05, 'epoch': 0.2}


  8%|▊         | 20/255 [03:42<43:00, 10.98s/it]

{'loss': 1.2192, 'grad_norm': 6.287754535675049, 'learning_rate': 4.607843137254902e-05, 'epoch': 0.39}


 12%|█▏        | 30/255 [05:29<40:36, 10.83s/it]

...

                                                 
100%|██████████| 255/255 [50:26<00:00, 10.29s/it]

{'eval_loss': 0.47604843974113464, 'eval_runtime': 41.7284, 'eval_samples_per_second': 2.444, 'eval_steps_per_second': 0.312, 'epoch': 5.0}


100%|██████████| 255/255 [50:28<00:00, 11.88s/it]

{'train_runtime': 3028.6235, 'train_samples_per_second': 0.669, 'train_steps_per_second': 0.084, 'train_loss': 0.3302167962725256, 'epoch': 5.0}


TrainOutput(global_step=255, training_loss=0.3302167962725256, metrics={'train_runtime': 3028.6235, 'train_samples_per_second': 0.669, 'train_steps_per_second': 0.084, 'total_flos': 532809454694400.0, 'train_loss': 0.3302167962725256, 'epoch': 5.0})

In [8]:
model.save_pretrained('./fine-tuned-Legalbert512')
tokenizer.save_pretrained('./fine-tuned-Legalbert512')
def classify_case(text):
    encoding = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**encoding)
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()
    return label_encoder.inverse_transform([predicted_class_id])[0]

example_text = '''M. JAGANNADHA RAO. J. This case concerns the proceedings arising out of an order of this Court dated 4.5.2000 proposing to re-open the quantum of punishments imposed in departmental inquiries on certain officers of the Delhi Development Authority (hereinafter called the DDA) who were connected with the land of the DDA allotted to M/s Skipper Construction Co. It was proposed to consider imposition of higher degree of punishments in view of the role of these officers in the said matter. After directions were given by this Court that disciplinary action be taken and punishments were imposed, this Court had no occasion to examine whether the right punishments were awarded to the officers in accordance with well known principles of law or whether the punishments required any upward revision.
The facts of the case limited to the present order are as follows :
By an order dated 29.11.94, this Court requested Justice O. Chinnappa Reddy (former Judge of this Court), to investigate into the conduct of the officials of the DDA including its ex-officio chairman at the relevant time, in handing over the possession of the suit land in M/s. Skipper Construction Pvt. Ltd. before receiving the auction amount in full and also in "conniving" at the construction thereon as well as at the advertisements given by it for bookings in the building in question. The learned Judge was also requested to "look into the legality and propriety of the order dated 4.10.98 passed by the then ex-officio Chairman of the DDA and the directions given by the Central Government under Section 41 of the Delhi Development Act."
Chairman to whom the file had been put up again on 2.6.1982 to confirm the telephonic instructions alleged to have been received from Baidwan nor was the matter brought to the notice of the Lt. Governor immediately as required under the Rules. The Union Public Service Commission had further noted that the Vice-Chairman, as a matter of fact, had stated before Justice Chinnappa Reddy Commission that he did not recollect exactly the conversation he had with Sri Baidwan on 28.5.1982.'''
prediction = classify_case(example_text)
print(f"Predicted Case Type: {prediction}")

Predicted Case Type: Civil Case


In [19]:
from transformers import BertTokenizer, BertForSequenceClassification
import joblib
tokenizer = BertTokenizer.from_pretrained('./fine-tuned-Legalbert512')
model = BertForSequenceClassification.from_pretrained('./fine-tuned-Legalbert512')
label_encoder = joblib.load('label_encoder.joblib')

In [23]:
def classify_case(text):
    encoding = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**encoding)
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()
    return label_encoder.inverse_transform([predicted_class_id])[0]

example_text = '''Facts: On the night of 5th September 2019, the accused, Dhananjay Singh, allegedly murdered his neighbor, Mr. Ravi Kumar, following a heated argument over a property dispute in Mumbai, Maharashtra. Witnesses reported hearing loud voices coming from the victim's house, followed by a gunshot. The police arrested Dhananjay Singh at the scene, where he was found holding a firearm.

Prosecution's Argument: The prosecution argued that Dhananjay Singh had a motive for murder due to the ongoing property dispute. They presented eyewitness testimonies that placed him at the scene and forensic evidence linking the firearm in his possession to the bullet found in the victim's body. The prosecution sought to prove that the act was premeditated and planned, making it a case of first-degree murder under Section 302 of the Indian Penal Code (IPC).

Defense's Argument: The defense contended that Dhananjay Singh acted in self-defense, claiming that the victim attacked him first with a knife, and in the struggle, the gun accidentally discharged. The defense argued that there was no intention to kill, and therefore, the charges should be reduced to culpable homicide not amounting to murder under Section 304 of the IPC.

Judgment: The court, after considering the evidence and witness testimonies, found Dhananjay Singh guilty of murder under Section 302 of the IPC. The court held that the defense's argument of self-defense was not supported by the evidence, and the killing was deliberate and intentional. The accused was sentenced to life imprisonment.'''
prediction = classify_case(example_text)
print(f"Predicted Case Type: {prediction}")

Predicted Case Type: Criminal Case
